<a href="https://colab.research.google.com/github/vapaul10/Portfolio-of-Valeri/blob/main/Copy_of_Amazon_Book_Order.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### Stratascratch tutorial (Valeri's work)

### Monthly Sales Rolling Average

###### Question 
You have been asked to calculate the rolling average for book sales in 2022.


A rolling average continuously updates a data set's average to include all data in the set up to that point. For example, the rolling average for February would be calculated by adding the book sales from January and February and dividing by two; the rolling average for March would be calculated by adding the book sales from January, February, and March and dividing by three; and so on.


Output the date, the sales for that date, and an extra column containing the rolling average rounded to the nearest whole number.

In [ ]:
#Install Python Datatable (same at data.table() in R)
#!pip install datatable
import datatable as dt

from datatable import (dt, f, by, ifelse, update, sort, count, min, max, mean, sum, rowsum)

ModuleNotFoundError: ignored

In [ ]:
# Import your libraries
import pandas as pd
import numpy as np
import datetime as datetimes
import time as time

In [ ]:
#Install python datatable which is the same as data.table() in R
!pip install datatables
import datatable as dt

from datatable import (dt, f, by, ifelse, update, sort, count,
                       min, max, mean, sum, rowsum)

SyntaxError: ignored

In [ ]:
# Install data from Amazon Books database Start writing code
amazon_books_pd = dt.fread("https://raw.githubusercontent.com/vapaul10/Portfolio-of-Valeri/41cefe2cd5218ab780ee4d3649a51b0fbf0f9947/amazon_books.csv")
#amazon_books_pd = pd.read_csv("https://raw.githubusercontent.com/vapaul10/Portfolio-of-Valeri/41cefe2cd5218ab780ee4d3649a51b0fbf0f9947/amazon_books.csv")

amazon_books_pd.head(10)

In [ ]:
## Data Attributes
amazon_books_pd.names

In [ ]:
## Install data from Book_Orders database. Dt  Start writing code
book_orders_pd = dt.fread("https://raw.githubusercontent.com/vapaul10/Portfolio-of-Valeri/main/book_orders.csv")
#book_orders_pd = pd.read_csv("https://raw.githubusercontent.com/vapaul10/Portfolio-of-Valeri/main/book_orders.csv")

book_orders_pd.head()

In [ ]:
amazon_books_pd = amazon_books_p.to_pandas()
amazon_books_pd.head

In [ ]:
book_orders_p=dt.Frame(book_orders_pd)

In [ ]:
book_orders_pd = book_orders_p.to_pandas()
book_orders_pd.head()

EXPLORATORY DATA ANALYSIS

In [ ]:
amazon_books_pd=amazon_books_p.to_pandas()  #Convert the Datatable object to Pandas object 
summary = amazon_books_pd.describe()
summary = summary.transpose()
summary

In [ ]:
book_orders_pd=book_orders_p.to_pandas()  #Convert the Datatable object to Pandas object 
summary = book_orders_pd.describe()
summary = summary.transpose()
summary

In [ ]:
#Factor the features of Amazon Books and Book Order databases. 
df = pd.merge(amazon_books_pd, book_orders_pd, on='book_id',how='inner').assign(sales =lambda x: x['unit_price']*x['quantity'])
df = df[pd.to_datetime(df['order_date']).dt.to_period('M').dt.year == 2022]
#df['order_date'] = df['order_date'].dt.month
df = df.groupby('order_date')['sales'].sum().reset_index().sort_values('order_date').assign(rolling_avg = lambda x: x['sales'].expanding().mean().round(0))
df.head()

In [ ]:
### Merge Data

In [ ]:
# merge data
merged = pd.merge(book_orders_pd, amazon_books_pd, on="book_id", how="inner")
merged["order_date"] = pd.to_datetime(merged["order_date"])
merged["order_month"] = merged["order_date"].dt.month
merged["year"] = merged["order_date"].dt.year
merged["sales"] = (merged["unit_price"]*merged["quantity"])
merged = merged.loc[(merged["year"] == 2022), :]
merged.head(15)


In [ ]:
### Result
result = (merged.groupby("order_month")["sales"]).sum().to_frame("monthly_sales").sort_values(by = "order_month").reset_index()
result["rolling_average"] = result["monthly_sales"].expanding().mean().round(0)
result